In [102]:
# !pip install -r requirements.txt 

In [103]:
# !wget https://raw.githubusercontent.com/milvus-io/milvus/master/deployments/docker/standalone/docker-compose.yml -O docker-compose.yml<br>
# !sudo docker-compose up -d

In [104]:
from pymilvus import connections
import psycopg2

from pymilvus import Collection, CollectionSchema, FieldSchema, DataType

from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.preprocessing import normalize

from pymilvus import Collection, CollectionSchema, FieldSchema, DataType

from flask import Flask, request
from flask_cors import CORS, cross_origin

from OpenSSL import SSL

In [105]:
import textract
import urllib.request
import re
import os
from collections import Counter
import operator

import xlrd
xlrd.xlsx.ensure_elementtree_imported(False, None)
xlrd.xlsx.Element_has_iter = True

## Class Test

---

In [165]:
class sparkai():
    
    def __init__(self, switch = True):

#         self.TABLE_NAME = "Skills_Embd"
#         self.TABLE_NAME_ = "Skills_Embd_"
#         self.field_name = "Skills"

        self.TABLE_NAME_1 = "Skills_Embd"
        self.TABLE_NAME_1_ = "Skills_Embd_"
        self.field_name_1 = "Skills"

        self.TABLE_NAME_2 = "Embds"
        self.TABLE_NAME_2_ = "Embds_"
        self.field_name_2 = "Mix_Fields"

        self.TABLE_NAME_3 = "Biz_Embds"
        self.TABLE_NAME_3_ = "Biz_Embds_"
        self.field_name_3 = "Biz"

        self.TABLE_NAME_4 = "Ppl_Embds"
        self.TABLE_NAME_4_ = "Ppl_Embds_"
        self.field_name_4 = "Ppl"

        self.TABLE_NAME_5 = "cmt_Embds"
        self.TABLE_NAME_5_ = "cmt_Embds_"
        self.field_name_5 = "cmt"

        self.TABLE_NAME_6 = "doc_Embds"
        self.TABLE_NAME_6_ = "doc_Embds_"
        self.field_name_6 = "doc"
        
        self.switch = switch
        
        self.connectdb(data = 'milvus')
        self.conn = self.connectdb(data = 'pg')
        
        self.model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
        self.cursor = self.conn.cursor()

        self.search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
        
    def switchit(self):
        
        if self.switch == True:
            self.switch = False
            return self.switch
        else:
            self.switch = True
            return self.switch
        
    ###############################################################
    ######### Connect Database 
    ###############################################################
        
    def connectdb(self, data = 'pg'):
    
        if data == 'milvus':
            connections.connect(host='104.196.170.44', port='19530')
   
        elif data == 'pg':
            conn = psycopg2.connect(database="alpha_alberta", host='35.203.122.7', port='5432', user='my_alpha_user', password='alpha_alberta@')
            print("CONNECTION ====", conn)
            return conn
        
    ###############################################################
    ######### Preprocess Text 
    ###############################################################
        
    def preprocess(self, text):   

        text = text.strip('\n')
        text = text.strip('\t')
        text = text.strip('\r')

        text = re.sub("\s+" , " ", text)
        text = text.strip()
        text = text.lower()

        return text
    
    ###############################################################
    ######### Recommendation & Student Recommendation Update 
    ###############################################################
    
    def update_rcmd(self):
    
        if self.switch == True:
            TN = self.TABLE_NAME_1_
        else:
            TN = self.TABLE_NAME_1

        try:
            collection = Collection(name = TN)
            collection.drop()
        except:
            pass

        pid = FieldSchema(name = "id", dtype = DataType.INT64, is_primary = True) #, auto_id=True)
        skills = FieldSchema(name = self.field_name_1, dtype = DataType.FLOAT_VECTOR, dim = 384)
        schema = CollectionSchema(fields = [pid, skills], description = "example collection")
        collection = Collection(name = TN, schema = schema)

        index_param = {
                "metric_type":"L2",
                "index_type":"IVF_SQ8",
                "params":{"nlist":1024}
            }
        collection.create_index(field_name=self.field_name_1, index_params=index_param)

    #   ['id', 'challenge_title', 'challenge_description', 'challenge_location', 'challenge_state', 
    #    'challenge_city', 'skills', 'status_type', 'post_type', 'is_active', 'is_archieve', 
    #    'is_student_required', 'created_at', 'challenge_end_date', 'updated_at', 
    #    'created_at_time', 'updated_at_time', 'company_name_id', 'user_id']

        try:
            self.cursor.execute("Select id, skills from challenge_creator_challengestatement;")
            rows_pos = self.cursor.fetchall()

            lst_pid, lst_skills = [], []
            for i, j in rows_pos:
                if j != '' and j != None:
                    lst_pid.append(i)
                    lst_skills.append(self.preprocess(j))

            if len(lst_skills) != 0:
                sentence_embeddings = self.model.encode(lst_skills)
                sentence_embeddings = normalize(sentence_embeddings)

                em = list(sentence_embeddings)
                mr = collection.insert([lst_pid, em])
                print('RCMD updated on', TN)
            else:
                print("There is no Data")
                
        except:
            print('query at update_rcmd')
            self.conn = self.connectdb(data = 'pg')
            
    ###############################################################
    ######### Recommendation Query 
    ###############################################################

    def query_rcmd(self, q = ''):

        if type(q) != str:
            raise Exception("q must be a string")

        search_skills = self.preprocess(q)

        embed = self.model.encode(search_skills)
        embed = embed.reshape(1,-1)
        embed = normalize(embed)
        query_embeddings = embed.tolist()
        

        if self.switch == True:
            collection = Collection(self.TABLE_NAME_1)
            collection.load()
        else:
            collection = Collection(self.TABLE_NAME_1_)
            collection.load()

        results = collection.search(query_embeddings, self.field_name_1, param=self.search_params, limit=1000, expr=None)

        recommendation1, recommendation2 = [], []
        for result in results[0]:
            
    #         print(result.distance)
    #         if result.distance < 1.50:

    #           ['id', 'challenge_title', 'challenge_description', 'challenge_location', 'challenge_state', 
    #            'challenge_city', 'skills', 'status_type', 'post_type', 'is_active', 'is_archieve', 
    #            'is_student_required', 'created_at', 'challenge_end_date', 'updated_at', 
    #            'created_at_time', 'updated_at_time', 'company_name_id', 'user_id']       

                try:
                    sql = "select * from challenge_creator_challengestatement where id = " + str(result.id) + ";"
                    self.cursor.execute(sql)
                    rows = self.cursor.fetchall()
#                 except:
#                     print('query at query_rcmd 1 ==>', result.id)
#                     self.connectdb(data = 'pg')

                    if len(rows) != 0:

                        if rows[0][11] == True and rows[0][6] != '':

                            if rows[0][-1] != None and rows[0][-1] != '':
                        
                                try:
                                    get_user_name = "select username from api_user where id =" + str(rows[0][-1]) + ";"
                                    self.cursor.execute(get_user_name)
                                    fetch_user_name = self.cursor.fetchall()
                                    fetch_user_name = fetch_user_name[0][0]
                                except:
                                    fetch_user_name = None
                                    print('query at query_rcmd 2 ==>', rows[0][-1])
                                    self.connectdb(data = 'pg')

                                try:
                                    get_profile_picture = "select profile_picture from api_personalinformation where id =" + str(rows[0][-1]) + ";"
                                    self.cursor.execute(get_profile_picture)
                                    fetch_profile_picture = self.cursor.fetchall()

                                    if len(fetch_profile_picture) != 0:
                                        fetch_profile_picture = '/dev_asp_bucket/' + fetch_profile_picture[0][0]
                                    else:
                                        fetch_profile_picture = ''
                                except:
                                    fetch_profile_picture = ''
                                    print('query at query_rcmd 3 ==>', rows[0][-1])
                                    self.connectdb(data = 'pg')

                            else:
                                fetch_user_name = None
                                fetch_profile_picture = ''

                            if rows[0][-2] != None and rows[0][-2] != '':

                                try:
                                    get_company_name = "select company_name from api_businessinformation where id =" + str(rows[0][-2]) + ";"
                                    self.cursor.execute(get_company_name)
                                    fetch_company_name = self.cursor.fetchall()
                                    fetch_company_name = fetch_company_name[0][0]
                                except:
                                    fetch_company_name = None
                                    print('query at query_rcmd 4 ==>', rows[0][-2])
                                    self.connectdb(data = 'pg')

                            else:
                                fetch_company_name = None

                            recommendation1.append({'id' : rows[0][0], 
                                                   'challenge_title': rows[0][1],
                                                   'challenge_description': rows[0][2],
                                                   'challenge_location': rows[0][3],
                                                   'challenge_state': rows[0][4],
                                                   'challenge_city': rows[0][5],
                                                   'skills': rows[0][6],
                                                   'status_type': rows[0][7],
                                                   'post_type': rows[0][8],
                                                   'is_active': rows[0][9],
                                                   'is_archieve': rows[0][10],
                                                   'is_student_required': rows[0][11],
                                                   'created_at': rows[0][12],
                                                   'challenge_end_date': rows[0][13],
                                                   'updated_at': rows[0][14],
                                                   'created_at_time': rows[0][15],
                                                   'updated_at_time': rows[0][16],
                                                   'company_name_id': rows[0][17],
                                                   'user_id': rows[0][18],
                                                   'user_name': fetch_user_name,
                                                   'company_name': fetch_company_name,
                                                   'profile_picture': fetch_profile_picture,
                                                   'distance': result.distance})


                        elif rows[0][11] == False and rows[0][6] != '':

                            if rows[0][-1] != None and rows[0][-1] != '':

                                try:
                                    get_user_name = "select username from api_user where id =" + str(rows[0][-1]) + ";"
                                    self.cursor.execute(get_user_name)
                                    fetch_user_name = self.cursor.fetchall()
                                    fetch_user_name = fetch_user_name[0][0]
                                except:
                                    fetch_user_name = None
                                    print('query at query_rcmd 5 ==>', rows[0][-1])
                                    self.connectdb(data = 'pg')

                                try:
                                    get_profile_picture = "select profile_picture from api_personalinformation where id =" + str(rows[0][-1]) + ";"
                                    self.cursor.execute(get_profile_picture)
                                    fetch_profile_picture = self.cursor.fetchall()

                                    if len(fetch_profile_picture) != 0:
                                        fetch_profile_picture = '/dev_asp_bucket/' + fetch_profile_picture[0][0]
                                    else:
                                        fetch_profile_picture = ''
                                except:
                                    fetch_profile_picture = ''
                                    print('query at query_rcmd 6 ==>', rows[0][-1])
                                    self.connectdb(data = 'pg')

                            else:
                                fetch_user_name = None
                                fetch_profile_picture = ''

                            if rows[0][-2] != None and rows[0][-2] != '':
                                
                                try:
                                    get_company_name = "select company_name from api_businessinformation where id =" + str(rows[0][-2]) + ";"
                                    self.cursor.execute(get_company_name)
                                    fetch_company_name = self.cursor.fetchall()
                                    fetch_company_name = fetch_company_name[0][0]
                                except:
                                    fetch_company_name = None
                                    print('query at query_rcmd 7 ==>', rows[0][-2])
                                    self.connectdb(data = 'pg')
                                    
                            else:
                                fetch_company_name = None

                            recommendation2.append({'id' : rows[0][0], 
                                                   'challenge_title': rows[0][1],
                                                   'challenge_description': rows[0][2],
                                                   'challenge_location': rows[0][3],
                                                   'challenge_state': rows[0][4],
                                                   'challenge_city': rows[0][5],
                                                   'skills': rows[0][6],
                                                   'status_type': rows[0][7],
                                                   'post_type': rows[0][8],
                                                   'is_active': rows[0][9],
                                                   'is_archieve': rows[0][10],
                                                   'is_student_required': rows[0][11],
                                                   'created_at': rows[0][12],
                                                   'challenge_end_date': rows[0][13],
                                                   'updated_at': rows[0][14],
                                                   'created_at_time': rows[0][15],
                                                   'updated_at_time': rows[0][16],
                                                   'company_name_id': rows[0][17],
                                                   'user_id': rows[0][18],
                                                   'user_name': fetch_user_name,
                                                   'company_name': fetch_company_name,
                                                   'profile_picture': fetch_profile_picture,
                                                   'distance': result.distance})

                except:
                    print('query at query_rcmd 1 ==>', result.id)
                    self.connectdb(data = 'pg')
                    
        recommendation = recommendation2 + recommendation1

        return recommendation
    
    ###############################################################
    ######### Student Recommendation Query 
    ###############################################################
                
    def query_stdrcmd(self):

        if type(q) != str:
            raise Exception("q must be a string")

        search_skills = self.preprocess(q)

        embed = self.model.encode(search_skills)
        embed = embed.reshape(1,-1)
        embed = normalize(embed)
        query_embeddings = embed.tolist()

        if self.switch == True:
            collection = Collection(self.TABLE_NAME_1)
            collection.load()
        else:
            collection = Collection(self.TABLE_NAME_1_)
            collection.load()

        results = collection.search(query_embeddings, self.field_name_1, param=self.search_params, limit=1000, expr=None)

        recommendation1, recommendation2 = [], []
        for result in results[0]:

    #         if result.distance < 1.50:

        #       ['id', 'challenge_title', 'challenge_description', 'challenge_location', 'challenge_state', 
        #        'challenge_city', 'skills', 'status_type', 'post_type', 'is_active', 'is_archieve', 
        #        'is_student_required', 'created_at', 'challenge_end_date', 'updated_at', 
        #        'created_at_time', 'updated_at_time', 'company_name_id', 'user_id']       

                try:
                    sql = "select * from challenge_creator_challengestatement AS ccc where id = " + str(result.id) + ";"
                    self.cursor.execute(sql)
                    rows = self.cursor.fetchall()
#                 except:
#                     print('query at query_stdrcmd 1')
#                     self.connectdb(data = 'pg')

                    if len(rows) != 0:

                        if rows[0][11] == True and rows[0][6] != '':

                            if rows[0][-1] != None and rows[0][-1] != '':
                    
                                try:
                                    get_user_name = "select username from api_user where id =" + str(rows[0][-1]) + ";"
                                    self.cursor.execute(get_user_name)
                                    fetch_user_name = self.cursor.fetchall()
                                    fetch_user_name = fetch_user_name[0][0]
                                except:
                                    fetch_user_name = None
                                    print('query at query_stdrcmd 2')
                                    self.connectdb(data = 'pg')

                                try:
                                    get_profile_picture = "select profile_picture from api_personalinformation where id =" + str(rows[0][-1]) + ";"
                                    self.cursor.execute(get_profile_picture)
                                    fetch_profile_picture = self.cursor.fetchall()

                                    if len(fetch_profile_picture) != 0:
                                        fetch_profile_picture = '/dev_asp_bucket/' + fetch_profile_picture[0][0]
                                    else:
                                        fetch_profile_picture = ''
                                except:
                                    fetch_profile_picture = ''
                                    print('query at query_stdrcmd 3')
                                    self.connectdb(data = 'pg')
                                    
                            else:
                                fetch_user_name = None
                                fetch_profile_picture = ''

                            if rows[0][-2] != None and rows[0][-2] != '':

                                try:
                                    get_company_name = "select company_name from api_businessinformation where id =" + str(rows[0][-2]) + ";"
                                    self.cursor.execute(get_company_name)
                                    fetch_company_name = self.cursor.fetchall()
                                    fetch_company_name = fetch_company_name[0][0]
                                except:
                                    fetch_company_name = None
                                    print('query at query_stdrcmd 4')
                                    self.connectdb(data = 'pg')

                            else:
                                fetch_company_name = None

                            recommendation1.append({'id' : rows[0][0], 
                                                   'challenge_title': rows[0][1],
                                                   'challenge_description': rows[0][2],
                                                   'challenge_location': rows[0][3],
                                                   'challenge_state': rows[0][4],
                                                   'challenge_city': rows[0][5],
                                                   'skills': rows[0][6],
                                                   'status_type': rows[0][7],
                                                   'post_type': rows[0][8],
                                                   'is_active': rows[0][9],
                                                   'is_archieve': rows[0][10],
                                                   'is_student_required': rows[0][11],
                                                   'created_at': rows[0][12],
                                                   'challenge_end_date': rows[0][13],
                                                   'updated_at': rows[0][14],
                                                   'created_at_time': rows[0][15],
                                                   'updated_at_time': rows[0][16],
                                                   'company_name_id': rows[0][17],
                                                   'user_id': rows[0][18],
                                                   'user_name': fetch_user_name,
                                                   'company_name': fetch_company_name,
                                                   'profile_picture': fetch_profile_picture})


                        elif rows[0][11] == False and rows[0][6] != '':

                            if rows[0][-1] != None and rows[0][-1] != '':

                                try:
                                    get_user_name = "select username from api_user where id =" + str(rows[0][-1]) + ";"
                                    self.cursor.execute(get_user_name)
                                    fetch_user_name = self.cursor.fetchall()
                                    fetch_user_name = fetch_user_name[0][0]
                                except:
                                    fetch_user_name = None
                                    print('query at query_stdrcmd 5')
                                    self.connectdb(data = 'pg')

                                try:
                                    get_profile_picture = "select profile_picture from api_personalinformation where id =" + str(rows[0][-1]) + ";"
                                    self.cursor.execute(get_profile_picture)
                                    fetch_profile_picture = self.cursor.fetchall()

                                    if len(fetch_profile_picture) != 0:
                                        fetch_profile_picture = '/dev_asp_bucket/' + fetch_profile_picture[0][0]
                                    else:
                                        fetch_profile_picture = ''
                                except:
                                    fetch_profile_picture = ''
                                    print('query at query_stdrcmd 6')
                                    self.connectdb(data = 'pg')
                                    
                            else:
                                fetch_user_name = None
                                fetch_profile_picture = ''

                            if rows[0][-2] != None and rows[0][-2] != '':
                                
                                try:
                                    get_company_name = "select company_name from api_businessinformation where id =" + str(rows[0][-2]) + ";"
                                    self.cursor.execute(get_company_name)
                                    fetch_company_name = self.cursor.fetchall()
                                    fetch_company_name = fetch_company_name[0][0]
                                except:
                                    fetch_company_name = None
                                    print('query at query_stdrcmd 7')
                                    self.connectdb(data = 'pg')
                                    
                            else:
                                fetch_company_name = None

                            recommendation2.append({'id' : rows[0][0], 
                                                   'challenge_title': rows[0][1],
                                                   'challenge_description': rows[0][2],
                                                   'challenge_location': rows[0][3],
                                                   'challenge_state': rows[0][4],
                                                   'challenge_city': rows[0][5],
                                                   'skills': rows[0][6],
                                                   'status_type': rows[0][7],
                                                   'post_type': rows[0][8],
                                                   'is_active': rows[0][9],
                                                   'is_archieve': rows[0][10],
                                                   'is_student_required': rows[0][11],
                                                   'created_at': rows[0][12],
                                                   'challenge_end_date': rows[0][13],
                                                   'updated_at': rows[0][14],
                                                   'created_at_time': rows[0][15],
                                                   'updated_at_time': rows[0][16],
                                                   'company_name_id': rows[0][17],
                                                   'user_id': rows[0][18],
                                                   'user_name': fetch_user_name,
                                                   'company_name': fetch_company_name,
                                                   'profile_picture': fetch_profile_picture})
                
                except:
                    print('query at query_stdrcmd 1')
                    self.connectdb(data = 'pg')
                
        recommendation = recommendation1 + recommendation2

        return recommendation
    
    ###############################################################
    ######### Search Update 
    ###############################################################
    
    def update_search(self):
    
        if self.switch == True:
            TN = self.TABLE_NAME_2_
        else:
            TN = self.TABLE_NAME_2

        try:
            collection = Collection(name = TN)
            collection.drop()
        except:
            pass

        pid = FieldSchema(name = "id", dtype = DataType.INT64, is_primary = True) #, auto_id=True)
        embds = FieldSchema(name = self.field_name_2, dtype = DataType.FLOAT_VECTOR, dim = 384)
        schema = CollectionSchema(fields = [pid, embds], description = "This table contails embeddings \
                                  of mixed fields i.e. id, challenge_title, challenge_description, \
                                  challenge_location and skills")
        collection = Collection(name = TN, schema = schema)

        index_param = {
                "metric_type":"L2",
                "index_type":"IVF_SQ8",
                "params":{"nlist":1024}
            }
        collection.create_index(field_name=self.field_name_2, index_params=index_param)

    #   ['id', 'challenge_title', 'challenge_description', 'challenge_location', 'challenge_state', 
    #    'challenge_city', 'skills', 'status_type', 'post_type', 'is_active', 'is_archieve', 
    #    'is_student_required', 'created_at', 'challenge_end_date', 'updated_at', 
    #    'created_at_time', 'updated_at_time', 'company_name_id', 'user_id']     

        try:
            self.cursor.execute("Select id, challenge_title, skills, challenge_location, challenge_state, challenge_city, \
                            challenge_description from challenge_creator_challengestatement")
            rows_pos = self.cursor.fetchall()

            lst_pid, lst_mix = [], []
            for i in rows_pos:

                lst_pid.append(i[0])

                s = ''
                for m in i[1:]:

                    if m is not None:
                        s = s + ' ' + str(m)

                lst_mix.append(self.preprocess(s))

            if len(lst_mix) != 0:
                sentence_embeddings = self.model.encode(lst_mix)
                sentence_embeddings = normalize(sentence_embeddings)

                em = list(sentence_embeddings)
                mr = collection.insert([lst_pid, em])
                print('SEARCH updated on', TN)
            else:
                print("There is no Data")
            
        except:
            print('query at update_search')
            self.connectdb(data = 'pg')
            
    ###############################################################
    ######### Search Query 
    ###############################################################
            
    def query_search(self, q = ''):
    
        if type(q) != str:
            raise Exception("q must be a string")

        search_skills = self.preprocess(q)

        embed = self.model.encode(search_skills)
        embed = embed.reshape(1,-1)
        embed = normalize(embed)
        query_embeddings = embed.tolist()

        if self.switch == True:
            collection = Collection(self.TABLE_NAME_2)
            collection.load()
        else:
            collection = Collection(self.TABLE_NAME_2_)
            collection.load()

        results = collection.search(query_embeddings, self.field_name_2, param=self.search_params, limit=1000, expr=None)

        search = []
        for result in results[0]:

    #         if result.distance < 1.8:

        #       ['id', 'challenge_title', 'challenge_description', 'challenge_location', 'challenge_state', 
        #        'challenge_city', 'skills', 'status_type', 'post_type', 'is_active', 'is_archieve', 
        #        'is_student_required', 'created_at', 'challenge_end_date', 'updated_at', 
        #        'created_at_time', 'updated_at_time', 'company_name_id', 'user_id']   

                try:
                    sql = "select * from challenge_creator_challengestatement where id = " + str(result.id) + ";"
                    self.cursor.execute(sql)
                    rows = self.cursor.fetchall()
#                 except:
#                     print('query at query_search 1')
#                     self.connectdb()

                    if len(rows) != 0:
        
                        if rows[0][-1] != None and rows[0][-1] != '':
                        
                            try:
                                get_user_name = "select username from api_user where id =" + str(rows[0][-1]) + ";"
                                self.cursor.execute(get_user_name)
                                fetch_user_name = self.cursor.fetchall()
                                fetch_user_name = fetch_user_name[0][0]
                            except:
                                fetch_user_name = None
                                print('query at query_search 2')
                                self.connectdb(data = 'pg')

                            try:
                                get_profile_picture = "select profile_picture from api_personalinformation where id =" + str(rows[0][-1]) + ";"
                                self.cursor.execute(get_profile_picture)
                                fetch_profile_picture = self.cursor.fetchall()

                                if len(fetch_profile_picture) != 0:
                                    fetch_profile_picture = '/dev_asp_bucket/' + fetch_profile_picture[0][0]
                                else:
                                    fetch_profile_picture = ''
                            except:
                                fetch_profile_picture = ''
                                print('query at query_search 3')
                                self.connectdb(data = 'pg')
                                
                        else:
                            fetch_user_name = None
                            fetch_profile_picture = ''

                        if rows[0][-2] != None and rows[0][-2] != '':

                            try:
                                get_company_name = "select company_name from api_businessinformation where id =" + str(rows[0][-2]) + ";"
                                self.cursor.execute(get_user_name)
                                fetch_company_name = self.cursor.fetchall()
                                fetch_company_name = fetch_company_name[0][0]
                            except:
                                fetch_company_name = None
                                print('query at query_search 4')
                                self.connectdb(data = 'pg')

                        else:
                            fetch_company_name = None

                        search.append({'id' : rows[0][0], 
                                       'challenge_title': rows[0][1],
                                       'challenge_description': rows[0][2],
                                       'challenge_location': rows[0][3],
                                       'challenge_state': rows[0][4],
                                       'challenge_city': rows[0][5],
                                       'skills': rows[0][6],
                                       'status_type': rows[0][7],
                                       'post_type': rows[0][8],
                                       'is_active': rows[0][9],
                                       'is_archieve': rows[0][10],
                                       'is_student_required': rows[0][11],
                                       'created_at': rows[0][12],
                                       'challenge_end_date': rows[0][13],
                                       'updated_at': rows[0][14],
                                       'created_at_time': rows[0][15],
                                       'updated_at_time': rows[0][16],
                                       'company_name_id': rows[0][17],
                                       'user_id': rows[0][18],
                                       'user_name': fetch_user_name,
                                       'company_name': fetch_company_name,
                                       'profile_picture': fetch_profile_picture})
                    
                except:
                    print('query at query_search 1')
                    self.connectdb(data = 'pg')
                    
        return search
    
    ###############################################################
    ######### Business Update 
    ###############################################################
    
    def update_biz(self):
    
        if self.switch == True:
            TN = self.TABLE_NAME_3_
        else:
            TN = self.TABLE_NAME_3

        try:
            collection = Collection(name = TN)
            collection.drop()
        except:
            pass

        pid = FieldSchema(name = "id", dtype = DataType.INT64, is_primary = True) #, auto_id=True)
        biz = FieldSchema(name = self.field_name_3, dtype = DataType.FLOAT_VECTOR, dim = 384)
        schema = CollectionSchema(fields = [pid, biz], description = "example collection")
        collection = Collection(name = TN, schema = schema)

        index_param = {
                "metric_type":"L2",
                "index_type":"IVF_SQ8",
                "params":{"nlist":1024}
            }
        collection.create_index(field_name=self.field_name_3, index_params=index_param)

    #   ['id', 'userid', 'company_name', 'company_website', 'company_description', 'company_email', 
    #    'company_phone', 'company_classification', 'company_type', 'gross_revenue', 'invite_through_company', 
    #    'years_in_business', 'start_date', 'end_date', 'created_at', 'modified_at', 'user_id', 'company_headcount']

    #   ['id', 'company_address_line_1', 'company_address_line_2', 'company_city', 'company_state', 
    #    'company_country', 'zip_code', 'created_at', 'modified_at', 'business_id', 'user_id']

        try: # company_classification,
            self.cursor.execute("Select id, company_name, company_description, company_type, \
                            company_website, company_email, company_phone from api_businessinformation")
            rows_pos = self.cursor.fetchall()

            lst_pid, lst_biz = [], []
            for i in rows_pos:

                lst_pid.append(i[0])

                s = ''
                for m in i[1:]:

                    if m is not None:
                        s = s + ' ' + str(m)

                try:
                    self.cursor.execute("Select company_address_line_1, company_address_line_2, company_city, \
                                    company_state, company_country, zip_code from api_businesslocation \
                                    WHERE business_id = {};".format(i[0]))
                    rows_loc = self.cursor.fetchall()

                    for j in rows_loc:
                        for n in j:

                            if n is not None:
                                s = s + ' ' + str(n)

                except:
                    print('query at update_biz 2')
                    self.connectdb(data = 'pg')

                lst_biz.append(self.preprocess(s))

            if len(lst_biz) != 0:
                sentence_embeddings = self.model.encode(lst_biz)
                sentence_embeddings = normalize(sentence_embeddings)

                em = list(sentence_embeddings)
                mr = collection.insert([lst_pid, em])
                print('BIZ updated on', TN)
            else:
                print("There is no Data")

        except:
            print('query at update_biz 1')
            self.connectdb(data = 'pg')
            
    ###############################################################
    ######### Business Query 
    ###############################################################
            
    def query_biz(self, q = ''):

        if type(q) != str:
            raise Exception("q must be a string")

        search_skills = self.preprocess(q)

        embed = self.model.encode(search_skills)
        embed = embed.reshape(1,-1)
        embed = normalize(embed)
        query_embeddings = embed.tolist()

        if self.switch == True:
            collection = Collection(self.TABLE_NAME_3)
            collection.load()
        else:
            collection = Collection(self.TABLE_NAME_3_)
            collection.load()

        results = collection.search(query_embeddings, self.field_name_3, param=self.search_params, limit=1000, expr=None)

        biz = []
        for result in results[0]:

    #         if result.distance < 1.5:

    #           ['id', 'userid', 'company_name', 'company_website', 'company_description', 'company_email', 
    #            'company_phone', 'company_classification', 'company_type', 'gross_revenue', 'invite_through_company', 
    #            'years_in_business', 'start_date', 'end_date', 'created_at', 'modified_at', 'user_id', 'company_headcount']

    #           ['id', 'company_address_line_1', 'company_address_line_2', 'company_city', 'company_state', 
    #            'company_country', 'zip_code', 'created_at', 'modified_at', 'business_id', 'user_id']

                try:
                    sql = "select * from api_businessinformation where id = " + str(result.id) + ";"
                    self.cursor.execute(sql)
                    rows = self.cursor.fetchall()
                except:
                    rows = []
                    print('query at query_biz 1')
                    self.connectdb(data = 'pg')

                try:
                    sql = "select * from api_businesslocation where business_id = " + str(result.id) + ";"
                    self.cursor.execute(sql)
                    rows_loc = self.cursor.fetchall()
                except:
                    rows_loc = []
                    print('query at query_biz 2')
                    self.connectdb(data = 'pg')

                loc = []
                if len(rows_loc) != 0:
                    for i in rows_loc:
                        loc.append({'id': i[0],
                                    'company_address_line_1': i[1],
                                    'company_address_line_2': i[2],
                                    'company_city': i[3],
                                    'company_state': i[4],
                                    'company_country': i[5],
                                    'zip_code': i[6],
                                    'created_at': i[7],
                                    'modified_at': i[8],
                                    'business_id': i[9],
                                    'user_id': i[10]})

                if len(rows) != 0:
                    biz.append({'id' : rows[0][0], 
                                'userid': rows[0][1],
                                'company_name': rows[0][2],
                                'company_website': rows[0][3],
                                'company_description': rows[0][4],
                                'company_email': rows[0][5],
                                'company_phone': rows[0][6],
    #                             'company_classification': rows[0][7],
                                'company_type': rows[0][7],
                                'gross_revenue': rows[0][8],
                                'invite_through_company': rows[0][9],
                                'years_in_business': rows[0][10],
                                'company_headcount': rows[0][11],
                                'start_date': rows[0][12],
                                'end_date': rows[0][13],
                                'created_at': rows[0][14],
                                'modified_at': rows[0][15],
                                'user_id': rows[0][16],
                                'locations': loc})

        return biz
    
    ###############################################################
    ######### People Update
    ###############################################################
    
    def update_ppl(self):

        if self.switch == True:
            TN = self.TABLE_NAME_4_
        else:
            TN = self.TABLE_NAME_4

        try:
            collection = Collection(name = TN)
            collection.drop()
        except:
            pass

        pid = FieldSchema(name = "id", dtype = DataType.INT64, is_primary = True) #, auto_id=True)
        ppl = FieldSchema(name = self.field_name_4, dtype = DataType.FLOAT_VECTOR, dim = 384)
        schema = CollectionSchema(fields = [pid, ppl], description = "example collection")
        collection = Collection(name = TN, schema = schema)

        index_param = {
                "metric_type":"L2",
                "index_type":"IVF_SQ8",
                "params":{"nlist":1024}
            }
        collection.create_index(field_name=self.field_name_4, index_params=index_param)

    #   ['id', 'first_name', 'last_name', 'personal_email', 'personal_skills', 'job_title', 'headline', 
    #    'bio', 'office_phone', 'personal_phone', 'address_line_1', 'address_line_2', 'profile_picture', 
    #    'city', 'state', 'country', 'zip_code', 'is_student', 'created_at', 'modified_at', 'user_id']

        try:
            self.cursor.execute("Select id, first_name, last_name, personal_skills, job_title, \
                            headline, bio, personal_phone, address_line_1, address_line_2, \
                            city, state, country, personal_email, office_phone, zip_code from api_personalinformation")
            rows_pos = self.cursor.fetchall()

            lst_pid, lst_ppl = [], []
            for i in rows_pos:

                lst_pid.append(i[0])

                s = ''
                for m in i[1:]:

                    if m is not None:
                        s = s + ' ' + str(m)

                lst_ppl.append(self.preprocess(s))

            if len(lst_ppl) != 0:
                sentence_embeddings = self.model.encode(lst_ppl)
                sentence_embeddings = normalize(sentence_embeddings)

                em = list(sentence_embeddings)
                mr = collection.insert([lst_pid, em])
                print('PPL updated on', TN)
            else:
                print("There is no Data")
                
        except:
            print('query at update_ppl')
            self.connectdb(data = 'pg')
            
    ###############################################################
    ######### People Query
    ###############################################################
            
    def query_ppl(self, q = ''):

        if type(q) != str:
            raise Exception("q must be a string")

        search_skills = self.preprocess(q)

        embed = self.model.encode(search_skills)
        embed = embed.reshape(1,-1)
        embed = normalize(embed)
        query_embeddings = embed.tolist()

        if self.switch == True:
            collection = Collection(self.TABLE_NAME_4)
            collection.load()
        else:
            collection = Collection(self.TABLE_NAME_4_)
            collection.load()

        results = collection.search(query_embeddings, self.field_name_4, param=self.search_params, limit=1000, expr=None)

        ppl = []
        for result in results[0]:   

    #         if result.distance < 1.5:

    #           ['id', 'first_name', 'last_name', 'personal_email', 'personal_skills', 'job_title', 'headline', 
    #            'bio', 'office_phone', 'personal_phone', 'address_line_1', 'address_line_2', 'profile_picture', 
    #            'city', 'state', 'country', 'zip_code', 'is_student', 'created_at', 'modified_at', 'user_id']

                try:
                    sql = "select * from api_personalinformation where id = " + str(result.id) + ";"
                    self.cursor.execute(sql)
                    rows = self.cursor.fetchall()
                    
                    if len(rows[0][12]) == 0:
                        pp = rows[0][12]
                    else:
                        pp = '/dev_asp_bucket/' + rows[0][12] 
                    
                except:
                    rows = []
                    print('query at query_ppl')
                    self.connectdb(data = 'pg')

                if len(rows) != 0:
                    ppl.append({'id' : rows[0][0], 
                                'first_name': rows[0][1],
                                'last_name': rows[0][2],
                                'personal_email': rows[0][3],
                                'personal_skills': rows[0][4],
                                'job_title': rows[0][5],
                                'headline': rows[0][6],
                                'bio': rows[0][7],
                                'office_phone': rows[0][8],
                                'personal_phone': rows[0][9],
                                'address_line_1': rows[0][10],
                                'address_line_2': rows[0][11],
                                'profile_picture': pp,
                                'city': rows[0][13], 
                                'state': rows[0][14], 
                                'country': rows[0][15], 
                                'zip_code': rows[0][16], 
                                'is_student': rows[0][17],
                                'created_at': rows[0][18], 
                                'modified_at': rows[0][19], 
                                'user_id': rows[0][20]})

        return ppl
    
    ###############################################################
    ######### Comment Update
    ###############################################################
    
    def update_cmt(self):

        if self.switch == True:
            TN = self.TABLE_NAME_5_
        else:
            TN = self.TABLE_NAME_5

        try:
            collection = Collection(name = TN)
            collection.drop()
        except:
            pass

        pid = FieldSchema(name = "id", dtype = DataType.INT64, is_primary = True) #, auto_id=True)
        oid = FieldSchema(name = "oid", dtype = DataType.INT64)
        cmt = FieldSchema(name = self.field_name_5, dtype = DataType.FLOAT_VECTOR, dim = 384)
        schema = CollectionSchema(fields = [pid, oid, cmt], description = "Comment embeddings with its object_id and id")
        collection = Collection(name = TN, schema = schema)

        index_param = {
                "metric_type":"L2",
                "index_type":"IVF_SQ8",
                "params":{"nlist":1024}
            }
        collection.create_index(field_name=self.field_name_5, index_params=index_param)

    #   ['id', 'object_id', 'user_comment', 'created_at', 'updated_at', 'created_at_time', 
    #    'updated_at_time', 'commented_by_id', 'company_name_id', 'content_type_id', 'parent_id']

        try:
            self.cursor.execute("Select id, object_id, user_comment from challenge_creator_comment")
            rows_pos = self.cursor.fetchall()

            lst_pid, lst_oid, lst_cmt = [], [], []
            for i in rows_pos:

                if i[2] != '' and i[2] != None:
                    lst_pid.append(i[0])  
                    lst_oid.append(i[1])
                    lst_cmt.append(self.preprocess(i[2]))

            if len(lst_cmt) != 0:
                sentence_embeddings = self.model.encode(lst_cmt)
                sentence_embeddings = normalize(sentence_embeddings)

                em = list(sentence_embeddings)
                mr = collection.insert([lst_pid, lst_oid, em])
                print('CMT updated on', TN)
            else:
                print("There is no Data")
    
        except:
            print('query at update_cmt')
            self.connectdb(data = 'pg')
            
    ###############################################################
    ######### Comment Query
    ###############################################################
    
    def query_cmt(self, q = ''):

        if type(q) != str:
            raise Exception("q must be a string")

        search_skills = self.preprocess(q)

        embed = self.model.encode(search_skills)
        embed = embed.reshape(1,-1)
        embed = normalize(embed)
        query_embeddings = embed.tolist()

        if self.switch == True:
            collection = Collection(self.TABLE_NAME_5)
            collection.load()
        else:
            collection = Collection(self.TABLE_NAME_5_)
            collection.load()

        results = collection.search(query_embeddings, self.field_name_5, param=self.search_params, limit=1000, expr=None)

        cmt = []
        for result in results[0]: 

    #         if result.distance < 1.5:

    #           ['id', 'object_id', 'user_comment', 'created_at', 'updated_at', 'created_at_time', 
    #            'updated_at_time', 'commented_by_id', 'company_name_id', 'content_type_id', 'parent_id']

    #           ['id', 'challenge_title', 'challenge_description', 'challenge_location', 'challenge_state', 
    #            'challenge_city', 'skills', 'status_type', 'post_type', 'is_active', 'is_archieve', 
    #            'is_student_required', 'created_at', 'challenge_end_date', 'updated_at', 
    #            'created_at_time', 'updated_at_time', 'company_name_id', 'user_id']  

                try:
                    cmt_sql = "select * from challenge_creator_comment where id = " + str(result.id) + ";"
                    self.cursor.execute(cmt_sql)
                    rows_cmt = self.cursor.fetchall()

                    if len(rows_cmt) != 0:
                        try:
                            sql = "select * from challenge_creator_challengestatement where id = {};".format(rows_cmt[0][1])
                            self.cursor.execute(sql)
                            rows = self.cursor.fetchall()
                        except:
                            rows = []
                            print('query at query_cmt 2')
                            self.connectdb(data = 'pg')

                        try:
                            sql = "select profile_picture from api_personalinformation where user_id = {};".format(rows_cmt[0][7])
                            self.cursor.execute(sql)
                            rows_pp = self.cursor.fetchall()
                            
                            if rows_pp[0][0] != '' and rows_pp[0][0] != None:
                                rows_pp = '/dev_asp_bucket/' + rows_pp[0][0]
                            else:
                                rows_pp = ''
                        except:
                            rows_pp = ''
                            print('query at query_cmt 3')
                            self.connectdb(data = 'pg')

                        if len(rows) != 0:
                            cmt.append({'id' : rows[0][0], 
                                        'challenge_title': rows[0][1],
                                        'challenge_description': rows[0][2],
                                        'challenge_location': rows[0][3],
                                        'challenge_state': rows[0][4],
                                        'challenge_city': rows[0][5],
                                        'skills': rows[0][6],
                                        'status_type': rows[0][7],
                                        'post_type': rows[0][8],
                                        'is_active': rows[0][9],
                                        'is_archieve': rows[0][10],
                                        'is_student_required': rows[0][11],
                                        'created_at': rows[0][12],
                                        'challenge_end_date': rows[0][13],
                                        'updated_at': rows[0][14],
                                        'created_at_time': rows[0][15],
                                        'updated_at_time': rows[0][16],
                                        'company_name_id': rows[0][17],
                                        'user_id': rows[0][18],
                                        'comment_id': rows_cmt[0][0],
                                        'user_comment': rows_cmt[0][2], 
                                        'profile_picture': rows_pp})
                            
                except:
                    print('query at query_cmt 1')
                    self.connectdb(data = 'pg')

        return cmt
    
    ###############################################################
    ######### File Metadata
    ###############################################################
    
    def fmeta(self, x):
    
        ### Download File ###

        fname = x.split('/')[-1]
        r = requests.get(x, stream=True)

        with open(fname, 'wb') as fd:
            fd.write(r.content)

        ############################################

        text = textract.process(fname) # x
        text = text.decode('utf8')

        text = self.preprocess(text)

        c = 0
        batch = ''
        batch_lst = []
        for i in text.split(' '):

            if c <= 15:
                batch = batch + ' ' + i
                c += 1

            else:
                batch_lst.append(batch.strip())
                batch = i
                c = 1

        batch_lst.append(batch.strip())

        if os.path.exists(fname):
            os.remove(fname)
        else:
            print("The file does not exist")

        return batch_lst
    
    ###############################################################
    ######### File Search Update
    ###############################################################
    
    def update_fsearch(self):
    
        if self.switch == True:
            TN = self.TABLE_NAME_6_
        else:
            TN = self.TABLE_NAME_6

        try:
            collection = Collection(name = TN)
            collection.drop()
        except:
            pass

        pid = FieldSchema(name = "id", dtype = DataType.INT64, is_primary = True) #, auto_id=True)
        uid = FieldSchema(name = "uid", dtype = DataType.INT64)
        did = FieldSchema(name = "did", dtype = DataType.INT64)
        fl = FieldSchema(name = self.field_name_6, dtype = DataType.FLOAT_VECTOR, dim = 384)
        schema = CollectionSchema(fields = [pid, uid, did, fl], description = "File Search")
        collection = Collection(name = TN, schema = schema)

        index_param = {
                "metric_type":"L2",
                "index_type":"IVF_SQ8",
                "params":{"nlist":1024}
            }
        collection.create_index(field_name = self.field_name_6, index_params = index_param)  

        try:
            self.cursor.execute("Select id, unique_id, document_id, mdata from api_filemeta;")
            rows_pos = self.cursor.fetchall()

            lst_pid, lst_uid, lst_did, lst_fl = [], [], [], []
            for i in rows_pos:

                if i[3] != '' and i[3] != None:
                    lst_pid.append(i[0])  
                    lst_uid.append(i[1])
                    lst_did.append(i[2])
                    lst_fl.append(i[3])

            if len(lst_fl) != 0:
                sentence_embeddings = self.model.encode(lst_fl)
                sentence_embeddings = normalize(sentence_embeddings)

                em = list(sentence_embeddings)
                mr = collection.insert([lst_pid, lst_uid, lst_did, em])
                print('FSEARCH updated on', TN)
            else:
                print("There is no Data")

        except:
            print('query at update_fsearch')
            self.connectdb(data = 'pg')
            
    ###############################################################
    ######### File Search Query
    ###############################################################
            
    def query_fsearch(self, q = ''):

        if type(q) != str:
            raise Exception("q must be a string")

        search_skills = q

        embed = self.model.encode(search_skills)
        embed = embed.reshape(1,-1)
        embed = normalize(embed)
        query_embeddings = embed.tolist()

        if self.switch == True:
            collection = Collection(self.TABLE_NAME_6)
            collection.load()
        else:
            collection = Collection(self.TABLE_NAME_6_)
            collection.load()

        results = collection.search(query_embeddings, self.field_name_6, param=self.search_params, limit=1000, expr=None)

        l = []
        for result in results[0]:
            if result.distance < 1.5:
                l.append(result.id)

        res = collection.query(expr = "id in {}".format(l), offset = 0, limit = 1000, 
                               output_fields = ["id", "uid", 'did'], consistency_level = "Strong")

        print(res)

        uid_lst_0, uid_lst_1, uid_lst_2 = [], [], []
        for i in res:

            if i['did'] == 0:
                uid_lst_0.append(i['uid'])
            if i['did'] == 1:
                uid_lst_1.append(i['uid'])
            if i['did'] == 2:
                uid_lst_2.append(i['uid'])
    #     print('='*50)
    #     print(uid_lst_0)
    #     print(uid_lst_1)
    #     print(uid_lst_2)

        c0 = Counter(uid_lst_0)
        c1 = Counter(uid_lst_1)
        c2 = Counter(uid_lst_2)

        sort = operator.itemgetter(1)
        sorted0 = sorted(c0.items(), key=sort, reverse=True)
        sorted1 = sorted(c1.items(), key=sort, reverse=True)
        sorted2 = sorted(c2.items(), key=sort, reverse=True)

        # business_id     lab_profile_id

        personal_doc = []
        for i in sorted0:
            try:
                print(i[0])
                d0_sql = "select upload_documents, user_id from api_personaldocumentupload where id = " + str(i[0]) + ";"
                self.cursor.execute(d0_sql)
                rows_d0 = self.cursor.fetchall()

    #             personal_doc.append(rows_d0)

                try:
                    sql_p = "select first_name, last_name from api_personalinformation where id = " + str(rows_d0[0][-1]) + ";"
                    self.cursor.execute(sql_p)
                    rows_p = self.cursor.fetchall()

                    fname_p = rows_p[0][0]
                    lname_p = rows_p[0][1]
                except:
                    fname_p = ''
                    lname_p = ''
                    print('query at query_fsearch 2')
                    self.connectdb(data = 'pg')

                personal_doc.append({'upload_documents': rows_d0[0][0],
                                    'user_id': rows_d0[0][1],
                                    'fname': fname_p,
                                    'lname': lname_p})

            except:
                print('query at query_fsearch 1')
                self.connectdb(data = 'pg')

        business_doc = []
        for i in sorted1:
            try:
                d1_sql = "select upload_documents, user_id, business_id from api_businessdocumentupload where id = " + str(i[0]) + ";"
                self.cursor.execute(d1_sql)
                rows_d1 = self.cursor.fetchall()
    #             personal_doc.append(rows_d1)

                try:
                    sql_b = "select company_name from api_businessinformation where id = " + str(rows_d1[0][-1]) + ";"
                    self.cursor.execute(sql_b)
                    rows_b = self.cursor.fetchall()

                    company_name = rows_b[0][0]
                except:
                    company_name = ''
                    print('query at query_fsearch 4')
                    self.connectdb(data = 'pg')

                business_doc.append({'upload_documents': rows_d1[0][0],
                                    'user_id': rows_d1[0][1],
                                    'company_name': company_name})

            except:
                print('query at query_fsearch 3')
                self.connectdb(data = 'pg')

        lab_doc = []
        for i in sorted2:
            try:
                d2_sql = "select upload_documents, user_id, lab_profile_id from lab_profile_labdocumentupload where id = " + str(i[0]) + ";"
                self.cursor.execute(d2_sql)
                rows_d2 = self.cursor.fetchall()
    #             personal_doc.append(rows_d2)

                try:
                    sql_d = "select company_name from lab_profile_labprofile where id = " + str(rows_d2[0][-1]) + ";"
                    self.cursor.execute(sql_d)
                    rows_d = self.cursor.fetchall()

                    lab_name = rows_d[0][0]
                except:
                    lab_name = ''
                    print('query at query_fsearch 6')
                    self.connectdb(data = 'pg')

                personal_doc.append({'upload_documents': rows_d2[0][0],
                                    'user_id': rows_d2[0][1],
                                    'lab_name': lab_name})

            except:
                print('query at query_fsearch 5')
                self.connectdb(data = 'pg')

    #     print(personal_doc)

        return {'personal_doc': personal_doc, 'business_doc': business_doc, 'lab_doc': lab_doc}

In [166]:
sai = sparkai(switch = True)

CONNECTION ==== <connection object at 0x7f673af667c0; dsn: 'user=my_alpha_user password=xxx dbname=alpha_alberta host=35.203.122.7 port=5432', closed: 0>


---

In [167]:
# %%time
# sai.update_fsearch()

In [168]:
# %%time
# # print(1, '-'*25, '>')
# rcmd = sai.query_cmt('new comment 3')
# for i, j in enumerate(rcmd):
    
#     print(i, "-->")
# #     print('Distance:', j['distance'])
#     print('='*25)
    
#     for m, n in j.items():
#         print(m, ':', n)
        
#     print('\n')

# #     if i == 1:
# #         break

In [169]:
# import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [170]:
# mdata = sai.fmeta('Ankitpatidar_TB_DJango_Flask.pdf')

# for i in mdata:
#     print(i)
#     print('#'*54)

In [174]:
# sai.switchit()

False

In [175]:
# doc = sai.query_fsearch('collabration is important because')

# for i, j in doc.items():
#     print(i, '==>') #, j)
#     print('='*50)
    
#     for l, k in enumerate(j):
#         print(l)
#         print('-'*50)
#         for m, n in k.items():
#             print(m, ':', n)
#     print('\n')
#     print('='*50)
#     print('='*50)
#     print('='*50)

#     print('\n')

---

## Update

In [ ]:
def update():

    try:
        sai.update_rcmd()
        print('RCMD UPDATED')
    except:
        print('Error in RCMD UPDATED')
    
    try:
        sai.update_stdrcmd()
        print('STDRCMD UPDATED')
    except:
        print('Error in STDRCMD UPDATED')

    try:
        sai.update_search()
        print('SEARCH UPDATED')
    except:
        print('Error in SEARCH UPDATED')

    try:
        sai.update_biz()
        print('BIZ UPDATED')
    except:
        print('Error in BIZ UPDATED')

    try:
        sai.update_ppl()
        print('PPL UPDATED')
    except:
        print('Error in PPL UPDATED')

    try:
        sai.update_cmt()
        print('CMT UPDATED')
    except:
        print('Error in CMT UPDATED')
       
    swt = sai.switchit()
        
    print('Switch is', swt)

---

In [ ]:
app = Flask(__name__)

In [ ]:
@app.route('/updt', methods=['GET'])
def updt():
    
    update()
        
    return ('', 204)

#===========================================#
    
@app.route('/get_data', methods=['GET'])
@cross_origin()
def get_data():
    
    args = request.args
    q = args.get('q')
    
    if type(q) != str:
        raise Exception("q must be a string")
        
    search = sai.query_search(q)
    biz = sai.query_biz(q)
    ppl = sai.query_ppl(q)
    cmt = sai.query_cmt(q)
    doc = sai.query_fsearch(q)
    
    return {'Search': search, 'Business': biz, 'People': ppl, 'Comment': cmt, 'Document_Search': doc}
    
@app.route('/get_rcmd', methods=['GET'])
@cross_origin()
def get_rcmd():
    
    args = request.args
    q = args.get('q')
    
    if type(q) != str:
        raise Exception("q must be a string")
        
    rcmd = sai.query_rcmd(q)
    
    return rcmd

@app.route('/get_stdrcmd', methods=['GET'])
@cross_origin()
def get_stdrcmd():
    
    args = request.args
    q = args.get('q')
    
    if type(q) != str:
        raise Exception("q must be a string")
        
    stdrcmd = sai.query_stdrcmd(q)
    
    return stdrcmd

@app.route('/get_fmeta', methods=['GET'])
@cross_origin()
def get_fmeta():
    
    args = request.args
    q = args.get('q')
    
    if type(q) != str:
        raise Exception("q must be a string")
        
    meta = sai.fmeta(q)
    
    return meta

@app.route('/get_fsearch', methods=['GET'])
@cross_origin()
def get_fsearch():
    
    args = request.args
    q = args.get('q')
    
    if type(q) != str:
        raise Exception("q must be a string")
        
    fsearch = sai.query_fsearch(q)
    
    return fsearch

In [ ]:
# from asgiref.wsgi import WsgiToAsgi

# import asyncio
# from hypercorn.config import Config
# from hypercorn.asyncio import serve

# from hypercorn.config import Config

# config = Config()
# config.bind = ["0.0.0.0:80"]

In [ ]:
if __name__ == '__main__':
    
    import logging
    logging.basicConfig(filename='error.log',level=logging.DEBUG)
    
    port = 80
    app.run(host='0.0.0.0', port=port, debug=True) # ssl_context=('cert.pem', 'key.pem')
#     asyncio.run(serve(app, config))

---

In [178]:
get_ipython().system('jupyter nbconvert text_search_engine_alberta_final_v1.ipynb --to python')

[NbConvertApp] Converting notebook text_search_engine_alberta_final_v1.ipynb to python
[NbConvertApp] Writing 70247 bytes to text_search_engine_alberta_final_v1.py


In [ ]:
# !pip install git+https://github.com/leukeleu/jupyter-linenumbers
# !jupyter nbextension install --py linenumbers --user
# !jupyter nbextension enable --py linenumbers --user

In [ ]:
# import psutil

# hdd = psutil.disk_usage('/')

# print ("Total GiB:", hdd.total / (10**9))
# print ("Used GiB:", hdd.used / (10**9))
# print ("Free GiB:", hdd.free / (10**9))

In [ ]:
# v = psutil.virtual_memory()
# print(v)
# total = v.total >> 30
# print(total)

In [ ]:
# 2112405504 >> 30